In [20]:
from astropy.io import fits
from specutils import extinction
import numpy as np
from extinction import ccm89, apply
import pyfits
import scipy
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib nbagg

In [29]:
# Define path and root name
path = '/Users/Laurie/'
name = 'M33-SW'

# Use coefficients to correct for the ME used for the flux calibration (Will be corrected automatically in the futur)
coeff3 = 0.8
coeff2 = 0.8
coeff1 = 0.6

ME3 = 0.869
ME2 = 0.814
ME1 = 0.690

# Type in the measured seeing for the covolution of the maps
# Use coefficient to correct for the flux calibration coefficient error (Will be corrected automatically in the futur)
#NW
#seeinga = 1/1.5 # arcsec seeing on SN3
#seeingb = 1/1.2 # arcsec seeing on SN2
#seeingoii = 1/1.1 # arcsec seeing on SN1
#cte3 = 1.0
#cte2 = 1.0
#cte1 = 1.0

#NE
#seeinga = 1/1.3 # arcsec seeing on SN3
#seeingb = 1/0.9 # arcsec seeing on SN2
#seeingoii = 1/0.8 # arcsec seeing on SN1
#cte3 = 1.0/1.137
#cte2 = 1.0
#cte1 = 1.0

#SW
seeinga = 1/1.2 # arcsec seeing on SN3
seeingb = 1/1.3 # arcsec seeing on SN2
seeingoii = 1/2.2 # arcsec seeing on SN1
cte3 = 0.902
cte2 = 1.0
cte1 = 1.0

#SE
#seeinga = 1/1.1 # arcsec seeing on SN3
#seeingb = 1/1.5 # arcsec seeing on SN2
#seeingoii = 1/1.7 # arcsec seeing on SN1
#cte3 = 1.824/1.08625
#cte2 = 1.0
#cte1 = 1.0

In [30]:
# import the aligned files.

flux_ha, hha = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6563.1x1.flux_align.fits', header=True)
flux_hb, hhb = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.4861.1x1.flux_align.fits', header=True)

flux_oii, hoii = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN1.1.0.LineMaps.map.3727.1x1.flux_align.fits', header=True)
flux_oiii1, hoiii1 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.4959.1x1.flux_align.fits', header=True)
flux_oiii2, hoiii2 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.5007.1x1.flux_align.fits', header=True)
flux_nii1, hnii1 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6548.1x1.flux_align.fits', header=True)
flux_nii2, hnii2 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6583.1x1.flux_align.fits', header=True)
flux_sii1, hsii1 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6716.1x1.flux_align.fits', header=True)
flux_sii2, hsii2 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6731.1x1.flux_align.fits', header=True)

flux_cont1, hcont1 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN1.1.0.LineMaps.map.all.1x1.height_align.fits', header=True)
flux_cont2, hcont2 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.all.1x1.height_align.fits', header=True)
flux_cont3, hcont3 = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.all.1x1.height_align.fits', header=True)

# import the aligned files error.

flux_ha_err, hflux_ha_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6563.1x1.flux-err_align.fits', header=True)
flux_hb_err, hflux_hb_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.4861.1x1.flux-err_align.fits', header=True)

flux_oii_err, hflux_oii_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN1.1.0.LineMaps.map.3727.1x1.flux-err_align.fits', header=True)
flux_oiii1_err, hflux_oiii1_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.4959.1x1.flux-err_align.fits', header=True)
flux_oiii2_err, hflux_oiii2_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.5007.1x1.flux-err_align.fits', header=True)
flux_nii1_err, hflux_nii1_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6548.1x1.flux-err_align.fits', header=True)
flux_nii2_err, hflux_nii2_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6583.1x1.flux-err_align.fits', header=True)
flux_sii1_err, hflux_sii1_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6716.1x1.flux-err_align.fits', header=True)
flux_sii2_err, hflux_sii2_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.6731.1x1.flux-err_align.fits', header=True)

flux_cont1_err, hcont1_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN1.1.0.LineMaps.map.all.1x1.height-err_align.fits', header=True)
flux_cont2_err, hcont2_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN2.1.0.LineMaps.map.all.1x1.height-err_align.fits', header=True)
flux_cont3_err, hcont3_err = pyfits.getdata(path+'Analysis/Aligned/'+name+'_SN3.1.0.LineMaps.map.all.1x1.height-err_align.fits', header=True)



In [31]:
# Correcting for flux calibration problem of the semets17B
flux_ha = flux_ha*coeff3/ME3/cte3
flux_hb = flux_hb*coeff2/ME2/cte2
flux_oii = flux_oii*coeff1/ME1/cte1
flux_oiii1 = flux_oiii1*coeff2/ME2/cte2
flux_oiii2 = flux_oiii2*coeff2/ME2/cte2
flux_nii1 = flux_nii1*coeff3/ME3/cte3
flux_nii2 = flux_nii2*coeff3/ME3/cte3
flux_sii1 = flux_sii1*coeff3/ME3/cte3
flux_sii2 = flux_sii2*coeff3/ME3/cte3
flux_cont1 = flux_cont1*coeff1/ME1/cte1
flux_cont2 = flux_cont2*coeff2/ME2/cte2
flux_cont3 = flux_cont3*coeff3/ME3/cte3

flux_ha_err = flux_ha_err*coeff3/ME3/cte3
flux_hb_err = flux_hb_err*coeff2/ME2/cte2
flux_oii_err = flux_oii_err*coeff1/ME1/cte1
flux_oiii1_err = flux_oiii1_err*coeff2/ME2/cte2
flux_oiii2_err = flux_oiii2_err*coeff2/ME2/cte2
flux_nii1_err = flux_nii1_err*coeff3/ME3/cte3
flux_nii2_err = flux_nii2_err*coeff3/ME3/cte3
flux_sii1_err = flux_sii1_err*coeff3/ME3/cte3
flux_sii2_err = flux_sii2_err*coeff3/ME3/cte3
flux_cont1_err = flux_cont1_err*coeff1/ME1/cte1
flux_cont2_err = flux_cont2_err*coeff2/ME2/cte2
flux_cont3_err = flux_cont3_err*coeff3/ME3/cte3


In [32]:
taille_pixels = 0.322

sigmaa = seeinga/taille_pixels/2.35 # gaussian sigma for the seeing convolution
sigmab = seeingb/taille_pixels/2.35 # gaussian sigma for the seeing convolution
sigmaoii = seeingoii/taille_pixels/2.35 # gaussian sigma for the seeing convolution

# Apply seeing convolution and consider a mean absorption profile of an old background stellar population (equivalent width in absorption of 2A for Ha and 4A for Hb)
flux_ha_seeing = scipy.ndimage.filters.gaussian_filter(flux_ha+flux_cont3*2, sigmaa, truncate=4.0)
flux_hb_seeing = scipy.ndimage.filters.gaussian_filter(flux_hb+flux_cont2*4, sigmab, truncate=4.0)

flux_ha_err_seeing = (scipy.ndimage.filters.gaussian_filter(((flux_ha_err)**2+(flux_cont3_err*2)**2), sigmaa**2, truncate=4.0))**0.5
flux_hb_err_seeing = (scipy.ndimage.filters.gaussian_filter(((flux_hb_err)**2+(flux_cont2_err*4)**2), sigmab**2, truncate=4.0))**0.5

# The Extinction formula using the theoritical Ha/Hb ratio   
EB_V = 2.5/1.07*np.log10(flux_ha_seeing/flux_hb_seeing/2.86)
EB_V_err = abs(2.5/1.07*np.log10(abs((flux_ha_seeing+flux_ha_err_seeing)/(flux_hb_seeing-flux_ha_err_seeing))/2.86) - 2.5/1.07*np.log10(abs((flux_ha_seeing-flux_ha_err_seeing)/(flux_hb_seeing+flux_ha_err_seeing))/2.86))/2
EB_V_err = EB_V*(flux_ha_err_seeing/flux_ha_seeing+flux_hb_err_seeing/flux_hb_seeing)

# The Av [mag] formula using the Rv value of 3.1
Av = EB_V*np.float(3.1) # Did not considered an error on the Rv value of 3.1
Av_err = EB_V_err*np.float(3.1) 


# Threshold to remove the low SNR pixels values for the Ha and Hb map. We out 0 for all pixels bellow a flux of 1e-17 erg s-1 cm-2 A-1
EB_V[(flux_ha_seeing < 5e-18)] = 0
EB_V[(flux_hb_seeing < 5e-18)] = 0
EB_V[(EB_V < 0)] = 0
EB_V_err[(flux_ha_seeing < 5e-18)] = 0
EB_V_err[(flux_hb_seeing < 5e-18)] = 0
EB_V_err[(EB_V < 0.0001)] = 0

Av[(flux_ha_seeing < 5e-18)] = 0
Av[(flux_hb_seeing < 5e-18)] = 0
Av[(Av < 0)] = 0
Av_err[(flux_ha_seeing < 5e-18)] = 0
Av_err[(flux_hb_seeing < 5e-18)] = 0
Av_err[(Av < 0.00001)] = 0

# Normal values are between 0 and 1 up to 4-5. 
# Write the B(B-V) file
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_EBV.fits', EB_V, header=hha, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_EBV-err.fits', EB_V_err, header=hha, clobber=True)


# Normal values are between 0 and 3 up to 10-15. 
# Write the Av file
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_Av.fits', Av, header=hha, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_Av-err.fits', Av_err, header=hha, clobber=True)

# Write the Ha/Hb ratio in a file for visual checkup
Hahb = flux_ha/flux_hb
Hahb[(flux_ha < 3e-17) | (flux_hb < 1e-17)] = 0

Hahb_err = ((flux_ha_err/flux_ha)**2+(flux_hb_err/flux_hb)**2)**0.5

# Should not be below 2.5 (hard to explain physically but maybe in some scattering dominated regions)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_Ha_Hb_ratio.fits', Hahb, header=hha, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_Ha_Hb_ratio-err.fits', Hahb_err, header=hha, clobber=True)


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning:

In [33]:
# Create arrays for the flux corrections and the error propagation.
flux_ha_corr = np.copy(flux_ha)
flux_hb_corr = np.copy(flux_hb)
flux_oii_corr = np.copy(flux_oii)
flux_oiii1_corr = np.copy(flux_oiii1)
flux_oiii2_corr = np.copy(flux_oiii2)
flux_nii1_corr = np.copy(flux_nii1)
flux_nii2_corr = np.copy(flux_nii2)
flux_sii1_corr = np.copy(flux_sii1)
flux_sii2_corr = np.copy(flux_sii2)
flux_cont1_corr = np.copy(flux_cont1)
flux_cont2_corr = np.copy(flux_cont2)
flux_cont3_corr = np.copy(flux_cont3)

flux_ha_corr_err = np.copy(flux_ha_err)
flux_hb_corr_err = np.copy(flux_hb_err)
flux_oii_corr_err = np.copy(flux_oii_err)
flux_oiii1_corr_err = np.copy(flux_oiii1_err)
flux_oiii2_corr_err = np.copy(flux_oiii2_err)
flux_nii1_corr_err = np.copy(flux_nii1_err)
flux_nii2_corr_err = np.copy(flux_nii2_err)
flux_sii1_corr_err = np.copy(flux_sii1_err)
flux_sii2_corr_err = np.copy(flux_sii2_err)
flux_cont1_corr_err = np.copy(flux_cont1_err)
flux_cont2_corr_err = np.copy(flux_cont2_err)
flux_cont3_corr_err = np.copy(flux_cont3_err)

# "deredden" flux by A_V to get the flux_corr

indice = np.where(Av > 0)

for k in range(0,len(indice[0])):
    
    i = indice[0][k]
    j = indice[1][k]
    flux_ha_corr[i,j] = apply(ccm89(np.array([6563.]), -Av[i,j], 3.1, unit='aa'), flux_ha[i,j])
    flux_hb_corr[i,j] = apply(ccm89(np.array([4861.]), -Av[i,j], 3.1, unit='aa'), flux_hb[i,j])
    flux_oii_corr[i,j] = apply(ccm89(np.array([3727.]), -Av[i,j], 3.1, unit='aa'), flux_oii[i,j])
    flux_oiii1_corr[i,j] = apply(ccm89(np.array([4959.]), -Av[i,j], 3.1, unit='aa'), flux_oiii1[i,j])
    flux_oiii2_corr[i,j] = apply(ccm89(np.array([5007.]), -Av[i,j], 3.1, unit='aa'), flux_oiii2[i,j])
    flux_nii1_corr[i,j] = apply(ccm89(np.array([6548.]), -Av[i,j], 3.1, unit='aa'), flux_nii1[i,j])
    flux_nii2_corr[i,j] = apply(ccm89(np.array([6583.]), -Av[i,j], 3.1, unit='aa'), flux_nii2[i,j])
    flux_sii1_corr[i,j] = apply(ccm89(np.array([6716.]), -Av[i,j], 3.1, unit='aa'), flux_sii1[i,j])
    flux_sii2_corr[i,j] = apply(ccm89(np.array([6731.]), -Av[i,j], 3.1, unit='aa'), flux_sii2[i,j])
    flux_cont1_corr[i,j] = apply(ccm89(np.array([3740.]), -Av[i,j], 3.1, unit='aa'), flux_cont1[i,j])
    flux_cont2_corr[i,j] = apply(ccm89(np.array([4950.]), -Av[i,j], 3.1, unit='aa'), flux_cont2[i,j])
    flux_cont3_corr[i,j] = apply(ccm89(np.array([6650.]), -Av[i,j], 3.1, unit='aa'), flux_cont3[i,j])
 
# Propagate error

    flux_ha_corr_err[i,j] = flux_ha_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_ha_err[i,j]/flux_ha[i,j]))
    flux_hb_corr_err[i,j] = flux_hb_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_hb_err[i,j]/flux_hb[i,j]))
    flux_oii_corr_err[i,j] = flux_oii_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_oii_err[i,j]/flux_oii[i,j]))
    flux_oiii1_corr_err[i,j] = flux_oiii1_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_oiii1_err[i,j]/flux_oiii1[i,j]))
    flux_oiii2_corr_err[i,j] = flux_oiii2_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_oiii2_err[i,j]/flux_oiii2[i,j]))
    flux_nii1_corr_err[i,j] = flux_nii1_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_nii1_err[i,j]/flux_nii1[i,j]))
    flux_nii2_corr_err[i,j] = flux_nii2_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_nii2_err[i,j]/flux_nii2[i,j]))
    flux_sii1_corr_err[i,j] = flux_sii1_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_sii1_err[i,j]/flux_sii1[i,j]))
    flux_sii2_corr_err[i,j] = flux_sii2_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_sii2_err[i,j]/flux_sii2[i,j]))
    flux_cont1_corr_err[i,j] = flux_cont1_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_cont1_err[i,j]/flux_cont1[i,j]))
    flux_cont2_corr_err[i,j] = flux_cont2_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_cont2_err[i,j]/flux_cont2[i,j]))
    flux_cont3_corr_err[i,j] = flux_cont3_corr[i,j]*((Av_err[i,j]/Av[i,j])+(flux_cont3_err[i,j]/flux_cont3[i,j]))

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in greater
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: divide by zero encountered in float_scalars
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:65: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in double_scalars


In [34]:
# Applying the seeing corrections to all maps and errors
flux_ha_corr_seeing = np.copy(flux_ha_corr)
flux_hb_corr_seeing = np.copy(flux_hb_corr)
flux_oii_corr_seeing = np.copy(flux_oii_corr)
flux_oiii1_corr_seeing = np.copy(flux_oiii1_corr)
flux_oiii2_corr_seeing = np.copy(flux_oiii2_corr)
flux_nii1_corr_seeing = np.copy(flux_nii1_corr)
flux_nii2_corr_seeing = np.copy(flux_nii2_corr)
flux_sii1_corr_seeing = np.copy(flux_sii1_corr)
flux_sii2_corr_seeing = np.copy(flux_sii2_corr)
flux_cont1_corr_seeing = np.copy(flux_cont1_corr)
flux_cont2_corr_seeing = np.copy(flux_cont2_corr)
flux_cont3_corr_seeing = np.copy(flux_cont3_corr)

flux_ha_corr_err_seeing = np.copy(flux_ha_corr_err)
flux_hb_corr_err_seeing = np.copy(flux_hb_corr_err)
flux_oii_corr_err_seeing = np.copy(flux_oii_corr_err)
flux_oiii1_corr_err_seeing = np.copy(flux_oiii1_corr_err)
flux_oiii2_corr_err_seeing = np.copy(flux_oiii2_corr_err)
flux_nii1_corr_err_seeing = np.copy(flux_nii1_corr_err)
flux_nii2_corr_err_seeing = np.copy(flux_nii2_corr_err)
flux_sii1_corr_err_seeing = np.copy(flux_sii1_corr_err)
flux_sii2_corr_err_seeing = np.copy(flux_sii2_corr_err)
flux_cont1_corr_err_seeing = np.copy(flux_cont1_corr_err)
flux_cont2_corr_err_seeing = np.copy(flux_cont2_corr_err)
flux_cont3_corr_err_seeing = np.copy(flux_cont3_corr_err)


flux_ha_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_ha_corr, sigmaa, truncate=4.0)
flux_hb_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_hb_corr, sigmab, truncate=4.0)
flux_oii_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_oii_corr, sigmaoii, truncate=4.0)
flux_oiii1_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_oiii1_corr, sigmab, truncate=4.0)
flux_oiii2_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_oiii2_corr, sigmab, truncate=4.0)
flux_nii1_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_nii1_corr, sigmaa, truncate=4.0)
flux_nii2_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_nii2_corr, sigmaa, truncate=4.0)
flux_sii1_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_sii1_corr, sigmaa, truncate=4.0)
flux_sii2_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_sii2_corr, sigmaa, truncate=4.0)
flux_cont1_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_cont1_corr, sigmaoii, truncate=4.0)
flux_cont2_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_cont2_corr, sigmab, truncate=4.0)
flux_cont3_corr_seeing = scipy.ndimage.filters.gaussian_filter(flux_cont3_corr, sigmaa, truncate=4.0)

# Propagate errors
flux_ha_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_ha_corr_err**2, sigmaa**2, truncate=4.0))**0.5
flux_hb_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_hb_corr_err**2, sigmab**2, truncate=4.0))**0.5
flux_oii_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_oii_corr_err**2, sigmaoii**2, truncate=4.0))**0.5
flux_oiii1_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_oiii1_corr_err**2, sigmab**2, truncate=4.0))**0.5
flux_oiii2_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_oiii2_corr_err**2, sigmab**2, truncate=4.0))**0.5
flux_nii1_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_nii1_corr_err**2, sigmaa**2, truncate=4.0))**0.5
flux_nii2_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_nii2_corr_err**2, sigmaa**2, truncate=4.0))**0.5
flux_sii1_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_sii1_corr_err**2, sigmaa**2, truncate=4.0))**0.5
flux_sii2_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_sii2_corr_err**2, sigmaa**2, truncate=4.0))**0.5
flux_cont1_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_cont1_corr_err**2, sigmaoii**2, truncate=4.0))**0.5
flux_cont2_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_cont2_corr_err**2, sigmab**2, truncate=4.0))**0.5
flux_cont3_corr_err_seeing = (scipy.ndimage.filters.gaussian_filter(flux_cont3_corr_err**2, sigmaa**2, truncate=4.0))**0.5

In [35]:
# Write the extinction corrected files

pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6563.1x1.flux.ext_corr.fits', flux_ha_corr_seeing, header=hha, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.4861.1x1.flux.ext_corr.fits', flux_hb_corr_seeing, header=hhb, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN1.1.0.LineMaps.map.3727.1x1.flux.ext_corr.fits', flux_oii_corr_seeing, header=hoii, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.4959.1x1.flux.ext_corr.fits', flux_oiii1_corr_seeing, header=hoiii1, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.5007.1x1.flux.ext_corr.fits', flux_oiii2_corr_seeing, header=hoiii2, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6548.1x1.flux.ext_corr.fits', flux_nii1_corr_seeing, header=hnii1, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6583.1x1.flux.ext_corr.fits', flux_nii2_corr_seeing, header=hnii2, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6716.1x1.flux.ext_corr.fits', flux_sii1_corr_seeing, header=hsii1, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6731.1x1.flux.ext_corr.fits', flux_sii2_corr_seeing, header=hsii2, clobber=True)

# Write the extinction corrected files errors

pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6563.1x1.flux-err.ext_corr.fits', flux_ha_corr_err_seeing, header=hflux_ha_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.4861.1x1.flux-err.ext_corr.fits', flux_hb_corr_err_seeing, header=hflux_hb_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN1.1.0.LineMaps.map.3727.1x1.flux-err.ext_corr.fits', flux_oii_corr_err_seeing, header=hflux_oii_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.4959.1x1.flux-err.ext_corr.fits', flux_oiii1_corr_err_seeing, header=hflux_oiii1_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.5007.1x1.flux-err.ext_corr.fits', flux_oiii2_corr_err_seeing, header=hflux_oiii2_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6548.1x1.flux-err.ext_corr.fits', flux_nii1_corr_err_seeing, header=hflux_nii1_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6583.1x1.flux-err.ext_corr.fits', flux_nii2_corr_err_seeing, header=hflux_nii2_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6716.1x1.flux-err.ext_corr.fits', flux_sii1_corr_err_seeing, header=hflux_sii1_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.6731.1x1.flux-err.ext_corr.fits', flux_sii2_corr_err_seeing, header=hflux_sii2_err, clobber=True)

/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN3.1.0.LineMaps.map.6563.1x1.flux.ext_corr.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN2.1.0.LineMaps.map.4861.1x1.flux.ext_corr.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN1.1.0.LineMaps.map.3727.1x1.flux.ext_corr.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN2.1.0.LineMaps.map.4959.1x1.flux.ext_corr.fits'.
  warnings.warn("Overwriting exis

In [36]:
# Write the continuum maps
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN1.1.0.LineMaps.map.all.1x1.height.ext_corr.fits', flux_cont1_corr_seeing, header=hcont1, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.all.1x1.height.ext_corr.fits', flux_cont2_corr_seeing, header=hcont2, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.all.1x1.height.ext_corr.fits', flux_cont3_corr_seeing, header=hcont3, clobber=True)
# Write the continuum maps errors
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN1.1.0.LineMaps.map.all.1x1.height-err.ext_corr.fits', flux_cont1_corr_err_seeing, header=hcont1_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN2.1.0.LineMaps.map.all.1x1.height-err.ext_corr.fits', flux_cont2_corr_err_seeing, header=hcont2_err, clobber=True)
pyfits.writeto(path+'Analysis/Dust_Extinction/'+name+'_SN3.1.0.LineMaps.map.all.1x1.height-err.ext_corr.fits', flux_cont3_corr_err_seeing, header=hcont3_err, clobber=True)

/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN1.1.0.LineMaps.map.all.1x1.height.ext_corr.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN2.1.0.LineMaps.map.all.1x1.height.ext_corr.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN3.1.0.LineMaps.map.all.1x1.height.ext_corr.fits'.
  warnings.warn("Overwriting existing file %r." % self.name)
/usr/local/lib/python2.7/site-packages/pyfits/file.py:372: UserWarning: Overwriting existing file '/Users/Laurie/Analysis/Dust_Extinction/M33-SW_SN1.1.0.LineMaps.map.all.1x1.height-err.ext_corr.fits'.
  warnings.warn("Overwrit